# **Decorators in Python**

### What Is a Decorator?

A **decorator** in Python is a callable (usually a function) that takes another function or method as an argument, wraps (or “decorates”) it in additional behavior, and returns a new callable. Decorators allow you to modify or extend the behavior of functions or methods **without** changing their source code.


###  Why Use Decorators?

- **Separation of Concerns**: Encapsulate cross‑cutting concerns (e.g., logging, timing, caching) in reusable wrappers.
- **DRY Principle**: Avoid repeating common code across multiple functions.
- **Readability**: Apply behavior with a clean, declarative `@decorator` syntax rather than manual wrapping.

---

### Functions as First‑Class Citizens

Before diving into decorators, recall that in Python:

- Functions can be **passed as arguments**  
- Functions can **return** other functions  
- Functions can be **assigned to variables**

In [1]:
def greet(name):
    return f"Hello, {name}"

In [ ]:
# assign function
say_hello = greet          
print(say_hello("Alice")) 

Hello, Alice


## Simple Decorator Pattern  
### 1. Writing the Decorator 

In [7]:
def my_decorator_function(func):
    def wrapper_fucntion():
        print("Before call")
        func()
        print("After call")
    return wrapper_fucntion

- **`func`**: original function passed in.  
- **`wrapper`**: new function adding behavior around `func()`.  

### 2. Applying Manually  

In [9]:
def say_hello():
    print("Hello!")

say_hello_display = my_decorator_function(say_hello)
say_hello_display()

Before call
Hello!
After call


## The `@` Syntax Sugar  
- Instead of manual reassignment, use `@my_decorator_function` above the function definition:

In [10]:
@my_decorator_function
def say_goodbye():
    print("Goodbye!")

say_goodbye()  # runs wrapper around original

Before call
Goodbye!
After call


In [12]:
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("James", "18")

display_info ran with arguments (James, 18)


### Let's try applying the our decorator `my_decorator_function` to the `display_info` function.

In [ ]:
@my_decorator_function
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("James", "18")

TypeError: my_decorator_function.<locals>.wrapper_fucntion() takes 0 positional arguments but 2 were given

**To avoid the above error we have to use the `*args`, and `**kwargs` argument in the wrpper function definition.**

Note: `args and kwargs` are just convention, you use any name.

In [17]:
def my_decorator_function(func):
    def wrapper(*args, **kwargs):
        print("wrapper executed this before {}".format(func.__name__))
        return func(*args, **kwargs)
    return wrapper

In [18]:
# Let's try again with the upfated function

@my_decorator_function
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("James", "18")

wrapper executed this before display_info
display_info ran with arguments (James, 18)


### Applications of Decorators

1. **Logging:** Create a function that automatically logs every call to a function—capturing its positional and keyword arguments.

In [ ]:
def my_logger(func):
    import logging
    # Configure logging: write INFO level logs to a file named after the function
    logging.basicConfig(
        filename=f'{func.__name__}.log',
        level=logging.INFO
    )

    def wrapper(*args, **kwargs):
        # Log the function call with its positional and keyword arguments
        logging.info(f'Ran with args: {args}, and kwargs: {kwargs}')
        # Call the original function and return its result
        return func(*args, **kwargs)

    return wrapper

##### **How it works:**  
  1. On decoration, configures Python’s logging to write to a file named after the function.  
  2. Wraps the original function in `wrapper()`.  
  3. Each time the function is called, `wrapper` logs the call details, then invokes the original function and returns its result.  

In [35]:
# use @my_logger as decorator

@my_logger
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("Johnson", "24")

display_info ran with arguments (Johnson, 24)


**2. Timing:** Decorators can also be used for timing how long a function run.

In [ ]:
def my_timer(func):
    import time

    def wrapper(*args, **kwargs):
        # Record the start time
        t1 = time.time()
        # Execute the original function and store its result
        result = func(*args, **kwargs)
        # Record the end time
        t2 = time.time()
        # Calculate and print the elapsed time
        print(f'{func.__name__} ran in: {t2 - t1:.4f} sec')
        # Return the original function’s result
        return result

    return wrapper

**How it works:**  

The `my_timer` decorator measures how long a function takes to execute. It wraps the original function in `wrapper()`, records the time immediately before and after the call, prints the elapsed duration, and then returns the function’s result, providing an easy way to benchmark any function by simply adding `@my_timer` above its definition.

In [ ]:
@my_timer
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("Johnson", "24")

display_info ran with arguments (Johnson, 24)
display_info ran in: 0.0001 sec


### How to apply multiple decorators to one function?

### **`Decorator stacking`**

**We can do that by stacking the decorators ontop of each other.**

In [ ]:
@my_timer
@my_logger
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("Daniel", "25")

display_info ran with arguments (Daniel, 25)
wrapper ran in: 0.0049 sec


### *It seems as though we are using these two decorators on separte functions*

In [41]:
@my_logger
@my_timer
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("Daniel", "25")

display_info ran with arguments (Daniel, 25)
display_info ran in: 0.0001 sec


### Observe the difference in the output as a result of the stacking arrangement?

1. **Implementation A:**
```
@my_timer
@my_logger
def func(...):
    ...
```

The above is equivalent to func = **`my_timer(my_logger(func))`**


2. **Implementation B:**
```
@my_logger
@my_logger
def func(...):
    ...
```

The above is equivalent to func = **`my_logger(my_logger(func))`**


**To solve all these confusion we can use `functools` package in python as follows**

In [42]:
from functools import wraps

# 1. Adding wrapper to logger

def my_logger(func):
    import logging
    # Configure logging: write INFO level logs to a file named after the function
    logging.basicConfig(
        filename=f'{func.__name__}.log',
        level=logging.INFO
    )

    @wraps(func)
    def wrapper(*args, **kwargs):
        # Log the function call with its positional and keyword arguments
        logging.info(f'Ran with args: {args}, and kwargs: {kwargs}')
        # Call the original function and return its result
        return func(*args, **kwargs)

    return wrapper


# 2. # Adding wrapper to Timer 

def my_timer(func):
    import time

    @wraps(func)
    def wrapper(*args, **kwargs):
        # Record the start time
        t1 = time.time()
        # Execute the original function and store its result
        result = func(*args, **kwargs)
        # Record the end time
        t2 = time.time()
        # Calculate and print the elapsed time
        print(f'{func.__name__} ran in: {t2 - t1:.4f} sec')
        # Return the original function’s result
        return result

    return wrapper

In [43]:
@my_logger
@my_timer
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("Jacob", "22")

display_info ran with arguments (Jacob, 22)
display_info ran in: 0.0001 sec


In [44]:
@my_timer
@my_logger
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("Jacob", "22")

display_info ran with arguments (Jacob, 22)
display_info ran in: 0.0002 sec


##### Now we can clearly see that the arrangements don't matter any more.
---
## Class‑Based Decorators  
- **Alternative** to function decorators: use a class with `__call__`:

In [19]:
class DecoratorClass:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print("Call method executed this before {}".format(self.func.__name__))
        return self.func(*args, **kwargs)

In [20]:
@DecoratorClass
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info("James", "18")

Call method executed this before display_info
display_info ran with arguments (James, 18)


In [22]:
@DecoratorClass
def say_goodbye():
    print("Goodbye!")

say_goodbye()

Call method executed this before say_goodbye
Goodbye!


## Common Real‑World Decorators  
- **`@staticmethod`** / **`@classmethod`** (built‑in)  
- **`@property`**, **`@cached_property`**  
- Third‑party: **`@login_required`**, **`@app.route`**, **`@lru_cache`**, **`@retry`**, **`@rate_limit`**, etc.